In [12]:
import csv
import json
import re
from bs4 import BeautifulSoup
from stop_words import get_stop_words
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
job_dict = {} #job_dict = {儲存每個jobtitle對應到的jobline有哪些}
jobTitle_list = [] #jobTile_list = [所有的jobtitle(包含重複值)]
normorlizedJobTitle_list = [] #normorlizedJobTitle_list = [所有前處理過後的jobtitle(包含重複值)]
normorlizedJobLine_list = [] #normorlizedJobLine_list = [所有前處理過後的jobline]

In [31]:
#前處理
def Preprocessing(job:str)->str:
    #只留下英文字
    # job = re.sub('/', ' ',job)
    
    job = re.sub('[^a-zA-Z]', ' ',job)
    job = re.sub(r'\b[a-zA-Z]\b', ' ',job)
    #將IT轉為Information Technology
    job = re.sub(r"IT ", "Information Technology ",job)
    
    #將所有英文字元都轉為小寫
    job = job.lower()
    #把文章前後的空白刪掉
    job = job.strip()
    #移除stopwords
    stopwords = get_stop_words('en')
    job = ' '.join([word for word in job.split() if word not in stopwords])
    #把有連續>=2個white space的地方改成一格就好
    job = re.sub(' +', ' ', job)
    #用Porter’s algorithm 來進行Stemming    
    ps = PorterStemmer()
    job = ' '.join([ps.stem(token) for token in job.split()])
            
    return job
     

In [5]:


# with open('./Resumes.csv', newline='', encoding="utf-8") as csvfile:

#   # 讀取 Resumes.csv 檔案內容(以dict的type來讀取)
#   rows = csv.DictReader(csvfile)
  
  
#   for row in rows:
#     #讀取Raw_html
#     html_content = row['Raw_html']
#     soup = BeautifulSoup(html_content, 'html.parser')
#     for job_title_tag in soup.find_all("span", class_="jobtitle"):
#       job_title = job_title_tag.get_text().strip()
      
#       #抓取屬於此jobtitle底下的jobline
#       div_tag = job_title_tag.find_parent("div", class_="singlecolumn")
#       job_desc_element = div_tag.find_all("span", class_="jobline")
#       # print(job_title)
#       # print(job_desc_element)
#       # print('.............................................................................')
      
#       #將jobtitle&jobline以字典的方式儲存下來，job_dict={'Software Engineer':[.....,.....,....]}
#       if job_desc_element:
#           job_desc_ul = job_desc_element[0].find("ul")
#           if job_desc_ul:
#               job_descs = [li.get_text().strip() for li in job_desc_ul.find_all("li")]
#               if job_title not in job_dict:
#                 job_dict[job_title] = job_descs
#               else:
#                 job_dict[job_title].extend(job_descs)


In [73]:

# with open('jobtitle_jobline_pairs.json', 'w', encoding='utf-8') as file:
#     # Write the dictionary in JSON format
#     json.dump(job_dict, file, ensure_ascii=False, indent=4)

經過人工手動合併一些太像的jobtitle後
jobtitle_jobline_pairs.json->jobtitle_jobline_pairs_merged.json

In [32]:
#讀取merged完成的jobtitle_jobline_pairs_merged.json
with open('jobtitle_jobline_pairs_merged.json', 'r', encoding='utf-8') as file:
    job_dict = json.load(file)

In [33]:
job_dict

{'Information Technology Manager': {'0': 'Expanded cross-functional organizational capacity by collaborating across departments on priorities, functions and common goals.',
  '1': 'Innovated SaaS, Mobile Apps, Continuity, and DR Services.',
  '2': "Responsible for all day-to-day operations and functions of Wilkes & McHugh's six offices and 300 staff members.",
  '3': 'Currently lead a team of 6 IT professionals and Developers in the planning, upgrades, and implementation of IT systems, business applications, and development of new software tools.',
  '4': 'Work closely with the CIO and senior management to ensure business and technology objectives are aligned.',
  '5': 'Researched new technologies and calculated future needs to achieve accurate capacity planning. Negotiated hardware and software purchases and maintenance contracts.',
  '6': "Dramatically reduced IT costs and maintenance by virtualizing the company's infrastructure using VMware.\xa0 Reduced physical servers from 50 to 4

將job_dict中所有jobline前面都加上對應的jobtitle，並全部併成一個檔:jobline_plus_jobtitle，使之後算分數的時候更容易找到相對高分的jobline

In [34]:
jobline_list = []
adjusted_jobline_list = []
for jobtitle, joblines in job_dict.items():
    for jobline in joblines.values():
        jobline_list.append(jobline)
        adjusted_jobline = jobtitle + ' ' +jobline
        adjusted_jobline_list.append(adjusted_jobline)

In [35]:
adjusted_jobline_list

['Information Technology Manager Expanded cross-functional organizational capacity by collaborating across departments on priorities, functions and common goals.',
 'Information Technology Manager Innovated SaaS, Mobile Apps, Continuity, and DR Services.',
 "Information Technology Manager Responsible for all day-to-day operations and functions of Wilkes & McHugh's six offices and 300 staff members.",
 'Information Technology Manager Currently lead a team of 6 IT professionals and Developers in the planning, upgrades, and implementation of IT systems, business applications, and development of new software tools.',
 'Information Technology Manager Work closely with the CIO and senior management to ensure business and technology objectives are aligned.',
 'Information Technology Manager Researched new technologies and calculated future needs to achieve accurate capacity planning. Negotiated hardware and software purchases and maintenance contracts.',
 "Information Technology Manager Drama

In [36]:

for job_title in job_dict.keys():
    jobTitle_list.append(job_title)
    normorlizedJobTitle = Preprocessing(job_title)
    normorlizedJobTitle_list.append(normorlizedJobTitle)
    
for job_line in adjusted_jobline_list:
    #normorlizedJobLine_list為了算tfidf用
    normorlizedJobLine_list.append(Preprocessing(job_line))
    

In [37]:
normorlizedJobLine_list

['inform technolog manag expand cross function organiz capac collabor across depart prioriti function common goal',
 'inform technolog manag innov saa mobil app continu dr servic',
 'inform technolog manag respons day day oper function wilk mchugh six offic staff member',
 'inform technolog manag current lead team inform technolog profession develop plan upgrad implement inform technolog system busi applic develop new softwar tool',
 'inform technolog manag work close cio senior manag ensur busi technolog object align',
 'inform technolog manag research new technolog calcul futur need achiev accur capac plan negoti hardwar softwar purchas mainten contract',
 'inform technolog manag dramat reduc inform technolog cost mainten virtual compani infrastructur use vmware reduc physic server streamlin disast recoveri process',
 'inform technolog manag plan execut activ directori domain migrat',
 'inform technolog manag coordin plan migrat mpl network site includ upgrad router switch network eq

In [38]:
dict_jobtitle = {
        'Index' : [i for i in range(0,len(jobTitle_list))],
        'JobTitle': jobTitle_list,
        'NormorlizedJobTitle': normorlizedJobTitle_list}

# 使用字典創建 DataFrame:(index,JobTitle,NormorlizedJobTitle)
dict_jobtitle = pd.DataFrame(dict_jobtitle)

In [39]:
dict_jobtitle

,Index,JobTitle,NormorlizedJobTitle
0,0,Information Technology Manager,inform technolog manag
1,1,Network Support Engineer,network support engin
2,2,Information Technology Technician,inform technolog technician
3,3,Software Engineer,softwar engin
4,4,Software Developer,softwar develop
...,...,...,...
225,225,"Information Technology Support Technician, IPT...",inform technolog support technician iptel subj...
226,226,Network Operations Analyst,network oper analyst
227,227,Technical Support Specialist,technic support specialist
228,228,Information Technology Trainee,inform technolog traine


In [41]:
dict_jobline = {
        'Index' : [i for i in range(0,len(jobline_list))],
        'JobLine': jobline_list,
        'NormorlizedJobLine': normorlizedJobLine_list}

# 使用字典創建 DataFrame:(index,JobTitle,NormorlizedJobTitle)
dict_jobline = pd.DataFrame(dict_jobline)

In [42]:
dict_jobtitle.iloc[3]

Index                                  3
JobTitle               Software Engineer
NormorlizedJobTitle        softwar engin
Name: 3, dtype: object

In [43]:
dict_jobtitle.to_csv('./indexTable_jobtitle.csv', encoding = 'utf-8',index = True)
dict_jobline.to_csv('./indexTable_jobline.csv', encoding = 'utf-8',index = True)

In [44]:

tfidf_vectorizer = TfidfVectorizer(smooth_idf=True)
tfidf_vector = tfidf_vectorizer.fit_transform(normorlizedJobTitle_list)

#查看词频结果  
tfidf_df_jobtitle = pd.DataFrame(tfidf_vector.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
# 建立 DF 字典
df_dict_jobtitle = {word: sum(1 for doc in normorlizedJobTitle_list if word in doc) for word in tfidf_vectorizer.get_feature_names_out()}


In [45]:
tfidf_vectorizer = TfidfVectorizer(smooth_idf=True)
tfidf_vector = tfidf_vectorizer.fit_transform(normorlizedJobLine_list)

#查看词频结果  
tfidf_df_jobline = pd.DataFrame(tfidf_vector.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
# 建立 DF 字典
df_dict_jobline = {word: sum(1 for doc in normorlizedJobLine_list if word in doc) for word in tfidf_vectorizer.get_feature_names_out()}

In [46]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth',None)
print(tfidf_df_jobline)

       aac  aadya   ab  ababa  abat  abbeel  abbi  abf  abil  abl  ...  zero  \
0      0.0    0.0  0.0    0.0   0.0     0.0   0.0  0.0   0.0  0.0  ...   0.0   
1      0.0    0.0  0.0    0.0   0.0     0.0   0.0  0.0   0.0  0.0  ...   0.0   
2      0.0    0.0  0.0    0.0   0.0     0.0   0.0  0.0   0.0  0.0  ...   0.0   
3      0.0    0.0  0.0    0.0   0.0     0.0   0.0  0.0   0.0  0.0  ...   0.0   
4      0.0    0.0  0.0    0.0   0.0     0.0   0.0  0.0   0.0  0.0  ...   0.0   
...    ...    ...  ...    ...   ...     ...   ...  ...   ...  ...  ...   ...   
10333  0.0    0.0  0.0    0.0   0.0     0.0   0.0  0.0   0.0  0.0  ...   0.0   
10334  0.0    0.0  0.0    0.0   0.0     0.0   0.0  0.0   0.0  0.0  ...   0.0   
10335  0.0    0.0  0.0    0.0   0.0     0.0   0.0  0.0   0.0  0.0  ...   0.0   
10336  0.0    0.0  0.0    0.0   0.0     0.0   0.0  0.0   0.0  0.0  ...   0.0   
10337  0.0    0.0  0.0    0.0   0.0     0.0   0.0  0.0   0.0  0.0  ...   0.0   

       zip   zo  zoho  zonal  zonar  zo

In [47]:
# Open a file for writing
with open('./jobtitle_dictionary.json', 'w', encoding='utf-8') as file:
    # Write the dictionary in JSON format
    json.dump(df_dict_jobtitle, file, ensure_ascii=False, indent=4)

In [48]:
# Open a file for writing
with open('./jobline_dictionary.json', 'w', encoding='utf-8') as file:
    # Write the dictionary in JSON format
    json.dump(df_dict_jobline, file, ensure_ascii=False, indent=4)

In [49]:
tfidf_df_jobtitle.to_csv('./jobtitle_tfidf.csv', encoding = 'utf-8',index = True)
tfidf_df_jobline.to_csv('./jobline_tfidf.csv', encoding = 'utf-8',index = True)